# Imports

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain


# Create Chat Instance 

In [4]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)


### Simple Chat

In [212]:
### Prompt template
template = """\
I am a {age} years old {gender}.\
I am going to do workout {days} times a week.\
These days should'nt contain rest day\
Each session takes {duration}.\
Provide me the best gym plan to maximise the {goal}.\
Your response should'nt contain extera information such as advices.

The template for the output is specify in between <<< >>>:

<<<
Day : Title
- move name : amount

Note :
>>>
"""

In [213]:
prompt_template = ChatPromptTemplate.from_template(template)

In [214]:
message = prompt_template.format_messages(age='26', gender='girl', days='3', duration='30 minutes', goal='fat burn')
response = chat(message)
print(response.content)

Day 1: Full Body Workout
- Squats: 3 sets of 12 reps
- Lunges: 3 sets of 12 reps (each leg)
- Push-ups: 3 sets of 12 reps
- Bent-over rows: 3 sets of 12 reps
- Plank: 3 sets of 30 seconds

Day 2: Cardio and Core
- Jumping jacks: 3 sets of 30 seconds
- Mountain climbers: 3 sets of 12 reps (each leg)
- High knees: 3 sets of 30 seconds
- Bicycle crunches: 3 sets of 12 reps (each side)
- Russian twists: 3 sets of 12 reps (each side)

Day 3: Upper Body Workout
- Dumbbell shoulder press: 3 sets of 12 reps
- Bicep curls: 3 sets of 12 reps
- Tricep dips: 3 sets of 12 reps
- Chest press: 3 sets of 12 reps
- Plank shoulder taps: 3 sets of 12 reps (each side)

Note: Remember to warm up before each workout and cool down/stretch afterwards.


In [215]:
message = prompt_template.format_messages(age='26', gender='girl', days='2', duration='30 minutes', goal='Muscle growth')
response = chat(message)
print(response.content)

Day 1: Upper Body Workout
- Bench Press: 3 sets of 8-10 reps
- Bent Over Rows: 3 sets of 8-10 reps
- Shoulder Press: 3 sets of 8-10 reps
- Bicep Curls: 3 sets of 8-10 reps
- Tricep Dips: 3 sets of 8-10 reps

Day 2: Lower Body Workout
- Squats: 3 sets of 8-10 reps
- Deadlifts: 3 sets of 8-10 reps
- Lunges: 3 sets of 8-10 reps
- Calf Raises: 3 sets of 8-10 reps
- Leg Press: 3 sets of 8-10 reps

Note: Make sure to warm up before each workout and cool down/stretch afterwards.


### Chained Chat

In [229]:
user_1 = {
    "age": "26",
    "gender": "girl",
    "days": "3",
    "duration": "30 minutes",
    "goal": "fat burn",
}

In [247]:
### Prompt template

template = """\
I am a {age} years old {gender}.\
I am going to do workout {days} times a week.\
These days should'nt contain rest day.\
Each session takes {duration}.\
Provide me the best gym plan to maximise the {goal}.\
Your response should'nt contain extera information such as advices.

The template for the output is specify in between <<< >>>:

<<<
Day : Title
- move name : amount

Note :
>>>
"""


prompt_template = ChatPromptTemplate.from_template(template)
# message = prompt_template.format_messages(age='26', gender='girl', days='2', duration='30 minutes', goal='Muscle growth')
chain_1 = LLMChain(llm=chat, prompt=prompt_template, output_key='plan')

In [248]:
print(chain_1.run(age=user_1['age'], gender=user_1['gender'], days=user_1['days'], duration=user_1['duration'], goal=user_1['goal']))

Day 1: Full Body Workout
- Squats: 3 sets of 12 reps
- Lunges: 3 sets of 12 reps (each leg)
- Push-ups: 3 sets of 12 reps
- Bent-over rows: 3 sets of 12 reps
- Plank: 3 sets of 30 seconds

Day 2: Cardio and Core
- Jumping jacks: 3 sets of 30 seconds
- Mountain climbers: 3 sets of 12 reps (each leg)
- High knees: 3 sets of 30 seconds
- Bicycle crunches: 3 sets of 12 reps (each side)
- Russian twists: 3 sets of 12 reps (each side)

Day 3: Upper Body Workout
- Dumbbell shoulder press: 3 sets of 12 reps
- Bicep curls: 3 sets of 12 reps
- Tricep dips: 3 sets of 12 reps
- Chest press: 3 sets of 12 reps
- Plank with shoulder taps: 3 sets of 12 reps (each side)

Note: Remember to warm up before each workout and cool down/stretch afterwards.


In [261]:
template_2 ="""\
Optimise the number of each set and its reps for the given workout plan in between <<<>>> for maximise {goal}.\
Be careful that the maximum duration of exercise for each session is {duration}\
The output template should be like the given plan.\
    
    
<<<{plan}>>>
"""

prompt_template_2 = ChatPromptTemplate.from_template(template_2)
chain_2 = LLMChain(llm=chat, prompt=prompt_template_2, output_key='final_plan')


In [262]:
overall_chain = SequentialChain(chains=[chain_1, chain_2], input_variables=['age', 'gender', 'days', 'duration', 'goal'],
                                 output_variables=['plan', 'final_plan'], verbose=True)

In [263]:
result = overall_chain(user_1)



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..



> Finished chain.


In [264]:
print(result['plan'])

Day 1: Full Body Workout
- Squats: 3 sets of 12 reps
- Lunges: 3 sets of 12 reps (each leg)
- Push-ups: 3 sets of 12 reps
- Bent-over rows: 3 sets of 12 reps
- Plank: 3 sets of 30 seconds

Day 2: Cardio and Core
- Jumping jacks: 3 sets of 30 seconds
- Mountain climbers: 3 sets of 12 reps (each leg)
- High knees: 3 sets of 30 seconds
- Bicycle crunches: 3 sets of 12 reps (each side)
- Russian twists: 3 sets of 12 reps (each side)

Day 3: Upper Body Workout
- Dumbbell shoulder press: 3 sets of 12 reps
- Bicep curls: 3 sets of 12 reps
- Tricep dips: 3 sets of 12 reps
- Chest press: 3 sets of 12 reps
- Plank with shoulder taps: 3 sets of 12 reps (each side)

Note: Remember to warm up before each workout and cool down/stretch afterwards.


In [265]:
print(result['final_plan'])

Day 1: Full Body Workout
- Squats: 3 sets of 12 reps
- Lunges: 3 sets of 12 reps (each leg)
- Push-ups: 3 sets of 12 reps
- Bent-over rows: 3 sets of 12 reps
- Plank: 3 sets of 30 seconds

Day 2: Cardio and Core
- Jumping jacks: 3 sets of 30 seconds
- Mountain climbers: 3 sets of 12 reps (each leg)
- High knees: 3 sets of 30 seconds
- Bicycle crunches: 3 sets of 12 reps (each side)
- Russian twists: 3 sets of 12 reps (each side)

Day 3: Upper Body Workout
- Dumbbell shoulder press: 3 sets of 12 reps
- Bicep curls: 3 sets of 12 reps
- Tricep dips: 3 sets of 12 reps
- Chest press: 3 sets of 12 reps
- Plank with shoulder taps: 3 sets of 12 reps (each side)
